**Reconocimiento automático de voz con una canalización**

Fuente: https://huggingface.co/learn/audio-course/chapter2/asr_pipeline

El reconocimiento automático de voz (ASR) es una tarea que implica transcribir grabaciones de audio de voz a texto. Esta tarea tiene numerosas aplicaciones prácticas, desde la creación de subtítulos para videos hasta la habilitación de comandos de voz para asistentes virtuales como Siri y Alexa.

En esta sección, usaremos la secuencia de comandos de reconocimiento automático de voz para transcribir una grabación de audio de una persona que hace una pregunta sobre el pago de una factura utilizando el mismo conjunto de datos MINDS-14.

Para comenzar, cargue el conjunto de datos y realice un muestreo ascendente a 16 kHz como se describe en Clasificación de audio con una secuencia de comandos.

Para transcribir una grabación de audio, podemos usar la secuencia de comandos de reconocimiento automático de voz de Transformers.

In [1]:
%%capture --no-stderr
%pip install datasets evaluate transformers[sentencepiece]
%pip install soundfile
%pip install torch

In [2]:
from datasets import load_dataset
from datasets import Audio

minds = load_dataset("PolyAI/minds14", name="es-ES", split="train", trust_remote_code=True)
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

d:\repos\GenIaEISEI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
%pip install keras
%pip install tf-keras
%pip install tensorflow
from transformers import pipeline

asr = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 11.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



A continuación, tomaremos un ejemplo del conjunto de datos y pasaremos sus datos sin procesar a la canalización:

In [9]:
%pip install librosa
example = minds[0]
asr(example["audio"]["array"])

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 22.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   ------- -------------------------------- 5.2/28.1 MB 26.5 MB/s eta 0:00:01
   --------------- ------------------------ 11.0/28.1 MB 27.5 MB/s eta 0:00:01
   ------------------------ --------------- 17.3/28.1 MB 28.7 MB/s eta 0:00:01
   ---------------------------------- ----- 24.1/28.1 MB 29.9 MB/s eta 0:00:01
   ---------------------------------------- 28.1/28.1 MB 27.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


d:\repos\GenIaEISEI\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input

{'text': ' Hola, buenas. A ver, tengo un problema con vuestra aplicación. Resulta que quiero hacer una transferencia bancaria a una cuenta conocida, pero me da error la aplicación. A ver qué puede ser.'}

Comparemos este resultado con la transcripción real para este ejemplo:

In [10]:
example["english_transcription"]

'hello good to see I have a problem with your application it turns out that I want to make a bank transfer to a known account'

¡El modelo parece haber hecho un buen trabajo al transcribir el audio! Solo se equivocó en una palabra ("card") en comparación con la transcripción original, lo cual es bastante bueno considerando que el hablante tiene acento australiano, donde la letra "r" a menudo no se pronuncia.

De manera predeterminada, esta secuencia utiliza un modelo entrenado para el reconocimiento automático de voz en inglés, lo cual está bien en este ejemplo. Si deseas intentar transcribir otros subconjuntos de MINDS-14 en un idioma diferente, puedes encontrar un modelo ASR entrenado previamente en el Hub. Puedes filtrar la lista de modelos primero por tarea y luego por idioma. Una vez que hayas encontrado el modelo que te gusta, pasa su nombre como argumento del modelo a la secuencia.

Probemos esto para la división alemana de MINDS-14. Carga el subconjunto "de-DE":

In [11]:
from datasets import load_dataset
from datasets import Audio

minds = load_dataset("PolyAI/minds14", name="de-DE", split="train")
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

Generating train split: 611 examples [00:00, 21486.89 examples/s]


Obten un ejemplo y vea cómo se supone que debe ser la transcripción:

In [9]:
example = minds[0]
example["transcription"]

'ich möchte gerne Geld auf mein Konto einzahlen'

Encuentre un modelo ASR entrenado previamente para el idioma alemán en el Hub, cree una instancia de canalización y transcriba el ejemplo:

In [10]:
from transformers import pipeline

asr = pipeline("automatic-speech-recognition", model="maxidl/wav2vec2-large-xlsr-german")
asr(example["audio"]["array"])

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at maxidl/wav2vec2-large-xlsr-german were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at maxidl/wav2vec2-large-xlsr-german and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

{'text': 'ich möchte gerne geld auf mein konto einzallen'}

Tips!

Cuando trabajes en la resolución de tu propia tarea, comienza con un pipeline simple como los que mostramos en esta unidad es una herramienta valiosa que ofrece varios beneficios:

Puede existir un modelo entrenado previamente que ya resuelva tu tarea muy bien, lo que te ahorra mucho tiempo
`pipeline()` se encarga de todo el procesamiento previo y posterior por ti, por lo que no tienes que preocuparte por poner los datos en el formato correcto para un modelo
si el resultado no es ideal, esto te brinda una línea de base rápida para futuros ajustes.